# Excel 分割器 - Windows SSO 版本

<!-- 
開發日期: 2025年1月
功能: 使用 Windows 整合驗證處理 SharePoint 分享
特色: 無需註冊 Azure AD 應用程式
-->

這個筆記本專為已登入 Windows 網域的使用者設計，可直接使用現有的 Windows 認證存取 SharePoint。

## 主要特色
- 🔐 使用 Windows 整合驗證（無需 Azure AD 應用程式）
- 📂 自動建立審查者專屬資料夾
- 📊 按審查者分割 Excel 檔案
- ✅ **保留資料驗證規則**
- 📤 批次 SharePoint 分享功能
- 🖥️ 專為 Windows 環境優化

## 系統需求
- Windows 作業系統
- 已登入公司網域的電腦
- Python 3.9 或更高版本
- SharePoint 存取權限
- Microsoft Office（建議）

## 步驟 1: 安裝必要套件

In [ ]:
# 安裝必要套件
import sys
import subprocess
import platform

# 檢查作業系統
if platform.system() != 'Windows':
    print("⚠️ 警告：此筆記本專為 Windows 設計，部分功能可能無法在其他系統上運作")

required_packages = [
    'pandas',
    'openpyxl',
    'ipywidgets',
    'requests',
    'requests-ntlm',     # Windows NTLM 認證
    'pywin32',          # Windows COM 介面
    'python-dotenv',
    'xlsxwriter',
    'Office365-REST-Python-Client'  # SharePoint 客戶端
]

print("🔍 檢查套件安裝狀態...")
for package in required_packages:
    try:
        if package == 'pywin32':
            __import__('win32com.client')
        elif package == 'Office365-REST-Python-Client':
            __import__('office365')
        else:
            __import__(package.replace('-', '_'))
        print(f"✓ {package} 已安裝")
    except ImportError:
        print(f"📦 正在安裝 {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✓ {package} 安裝完成")

print("\n✅ 所有必要套件已就緒！")

## 步驟 2: 匯入函式庫與設定

In [ ]:
import os
import shutil
import pandas as pd
from pathlib import Path
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import glob
from datetime import datetime
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import requests
from requests_ntlm import HttpNtlmAuth
import win32com.client
import getpass
import time
from typing import Dict, List, Optional, Tuple
import re
from dotenv import load_dotenv
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.authentication_context import AuthenticationContext

# 載入環境變數
load_dotenv()

# 檢查 tkinter (檔案對話框)
try:
    import tkinter as tk
    from tkinter import filedialog, messagebox
    TKINTER_AVAILABLE = True
except ImportError:
    TKINTER_AVAILABLE = False

print("✓ 函式庫匯入成功")
print(f"✓ 使用者: {getpass.getuser()}")
print(f"✓ 電腦名稱: {os.environ.get('COMPUTERNAME', '未知')}")
print(f"✓ 網域: {os.environ.get('USERDOMAIN', '未知')}")

# 全域變數
sharepoint_ctx = None
current_user = getpass.getuser()
site_url = None

## 步驟 3: Windows 整合驗證功能

In [ ]:
def connect_sharepoint_windows_auth(site_url: str) -> Optional[ClientContext]:
    """使用 Windows 整合驗證連接 SharePoint"""
    try:
        # 方法 1: 使用當前 Windows 認證
        ctx = ClientContext(site_url)
        
        # 嘗試使用 Windows 認證
        ctx.with_credentials(UserCredential("", ""))  # 空白表示使用當前 Windows 使用者
        
        # 測試連線
        web = ctx.web
        ctx.load(web)
        ctx.execute_query()
        
        print(f"✅ 成功連接到 SharePoint: {web.properties['Title']}")
        return ctx
        
    except Exception as e:
        print(f"❌ Windows 認證失敗: {str(e)}")
        print("嘗試使用互動式登入...")
        
        # 方法 2: 互動式登入視窗
        try:
            # 建立認證對話框
            root = tk.Tk()
            root.withdraw()
            
            # 使用 Windows 認證對話框
            from tkinter import simpledialog
            
            username = simpledialog.askstring(
                "SharePoint 登入",
                "使用者名稱 (domain\\username):",
                initialvalue=f"{os.environ.get('USERDOMAIN', '')}\\{current_user}"
            )
            
            if not username:
                return None
                
            password = simpledialog.askstring(
                "SharePoint 登入",
                "密碼:",
                show='*'
            )
            
            root.destroy()
            
            if not password:
                return None
            
            # 使用提供的認證
            ctx = ClientContext(site_url)
            ctx.with_credentials(UserCredential(username, password))
            
            # 測試連線
            web = ctx.web
            ctx.load(web)
            ctx.execute_query()
            
            print(f"✅ 成功連接到 SharePoint: {web.properties['Title']}")
            return ctx
            
        except Exception as e2:
            print(f"❌ 互動式登入失敗: {str(e2)}")
            return None

def test_sharepoint_connection(ctx: ClientContext) -> bool:
    """測試 SharePoint 連線"""
    try:
        web = ctx.web
        ctx.load(web)
        ctx.execute_query()
        return True
    except:
        return False

def get_current_user_info(ctx: ClientContext) -> dict:
    """取得當前使用者資訊"""
    try:
        user = ctx.web.current_user
        ctx.load(user)
        ctx.execute_query()
        
        return {
            'title': user.properties.get('Title', ''),
            'email': user.properties.get('Email', ''),
            'login_name': user.properties.get('LoginName', '')
        }
    except:
        return {'title': current_user, 'email': '', 'login_name': ''}

print("✓ Windows 整合驗證功能已就緒")

## 步驟 4: SharePoint 操作功能

In [ ]:
def sanitize_folder_name(name: str) -> str:
    """清理資料夾名稱，確保相容性"""
    # 移除或替換不允許的字元
    invalid_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|', '#', '%']
    sanitized = name.strip()
    
    for char in invalid_chars:
        sanitized = sanitized.replace(char, '_')
    
    # 限制長度
    if len(sanitized) > 255:
        sanitized = sanitized[:255].rstrip()
    
    return sanitized

def create_sharepoint_folder(ctx: ClientContext, folder_path: str) -> bool:
    """在 SharePoint 建立資料夾"""
    try:
        # 取得文件庫
        web = ctx.web
        
        # 使用預設的「文件」庫
        folder = web.folders.add(f"Shared Documents/{folder_path}")
        ctx.execute_query()
        
        print(f"✓ 已建立資料夾: {folder_path}")
        return True
        
    except Exception as e:
        if "already exists" in str(e).lower():
            print(f"✓ 資料夾已存在: {folder_path}")
            return True
        else:
            print(f"❌ 建立資料夾失敗: {str(e)}")
            return False

def upload_file_to_folder(ctx: ClientContext, local_file_path: str, 
                         sharepoint_folder: str) -> bool:
    """上傳檔案到 SharePoint 資料夾"""
    try:
        with open(local_file_path, 'rb') as f:
            file_content = f.read()
        
        file_name = os.path.basename(local_file_path)
        target_folder = ctx.web.get_folder_by_server_relative_url(
            f"Shared Documents/{sharepoint_folder}"
        )
        
        target_folder.upload_file(file_name, file_content).execute_query()
        
        print(f"✓ 已上傳: {file_name}")
        return True
        
    except Exception as e:
        print(f"❌ 上傳失敗: {str(e)}")
        return False

def share_folder_with_email(ctx: ClientContext, folder_path: str, 
                           email: str, permission_level: str = 'Edit') -> bool:
    """分享資料夾給特定 email"""
    try:
        # 取得資料夾
        folder = ctx.web.get_folder_by_server_relative_url(
            f"Shared Documents/{folder_path}"
        )
        
        # 使用 SharePoint 的分享功能
        # 注意：這需要適當的權限設定
        sharing_result = folder.share_link(
            sharing_link_kind=2,  # 編輯連結
            share_with=email
        )
        ctx.execute_query()
        
        print(f"✓ 已分享給: {email}")
        return True
        
    except Exception as e:
        print(f"❌ 分享失敗給 {email}: {str(e)}")
        # 嘗試替代方法
        try:
            # 使用權限設定
            folder = ctx.web.get_folder_by_server_relative_url(
                f"Shared Documents/{folder_path}"
            )
            
            # 中斷繼承
            folder.list_item_all_fields.break_role_inheritance(True, False)
            ctx.execute_query()
            
            # 授予權限
            user = ctx.web.ensure_user(email)
            ctx.execute_query()
            
            role_def = ctx.web.role_definitions.get_by_name(permission_level)
            folder.list_item_all_fields.role_assignments.add_role_assignment(
                user, role_def
            )
            ctx.execute_query()
            
            print(f"✓ 已使用權限方式分享給: {email}")
            return True
            
        except Exception as e2:
            print(f"❌ 替代分享方法也失敗: {str(e2)}")
            return False

print("✓ SharePoint 操作功能已就緒")

## 步驟 5: Excel 處理功能（使用方案 2 - 複製檔案後處理）

In [ ]:
def find_column(worksheet, column_name):
    """在工作表中尋找欄位"""
    for col_idx, cell in enumerate(worksheet[1], start=1):
        if cell.value == column_name:
            return col_idx
    raise ValueError(f"找不到 '{column_name}' 欄位！")

def copy_selected_documents(source_dir, dest_dir, copy_word=True, copy_pdf=True):
    """複製選定的文件類型"""
    copied_files = []
    
    if copy_word:
        word_patterns = [
            os.path.join(source_dir, "*.docx"),
            os.path.join(source_dir, "*.doc")
        ]
        
        for pattern in word_patterns:
            for file in glob.glob(pattern):
                if os.path.isfile(file):
                    dest_path = os.path.join(dest_dir, os.path.basename(file))
                    shutil.copy2(file, dest_path)
                    copied_files.append(os.path.basename(file))
    
    if copy_pdf:
        pdf_pattern = os.path.join(source_dir, "*.pdf")
        for file in glob.glob(pdf_pattern):
            if os.path.isfile(file):
                dest_path = os.path.join(dest_dir, os.path.basename(file))
                shutil.copy2(file, dest_path)
                copied_files.append(os.path.basename(file))
    
    return copied_files

def process_reviewer_excel_copy_first(file_path, reviewer, column_name, output_folder):
    """使用方案 2: 先複製整個檔案再處理（保留資料驗證）"""
    try:
        # 清理審查者名稱
        reviewer_name = sanitize_folder_name(str(reviewer).strip())
        
        # 建立審查者資料夾
        reviewer_folder = os.path.join(output_folder, reviewer_name)
        os.makedirs(reviewer_folder, exist_ok=True)
        
        # 建立新檔名
        base_name = os.path.basename(file_path)
        name_without_ext = os.path.splitext(base_name)[0]
        ext = os.path.splitext(base_name)[1]
        new_filename = f"{name_without_ext} - {reviewer_name}{ext}"
        dst_path = os.path.join(reviewer_folder, new_filename)
        
        # 先複製整個檔案
        shutil.copy2(file_path, dst_path)
        print(f"  ✓ 已複製檔案: {new_filename}")
        
        # 使用 openpyxl 處理複製的檔案
        wb = load_workbook(dst_path, data_only=False, keep_vba=True, keep_links=True)
        main_ws = wb.active
        
        # 保留所有資料驗證資訊
        data_validations = list(main_ws.data_validations.dataValidation)
        
        # 尋找並刪除非相關列
        col_idx = find_column(main_ws, column_name)
        rows_to_delete = []
        
        for row in range(2, main_ws.max_row + 1):
            cell_value = main_ws.cell(row=row, column=col_idx).value
            if str(cell_value) != str(reviewer):
                rows_to_delete.append(row)
        
        print(f"  ✓ 找到 {len(rows_to_delete)} 列需要刪除")
        
        # 從底部開始刪除
        for row in sorted(rows_to_delete, reverse=True):
            main_ws.delete_rows(row)
        
        # 重新應用資料驗證
        for dv in data_validations:
            main_ws.add_data_validation(dv)
        
        # 儲存變更
        wb.save(dst_path)
        wb.close()
        
        print(f"  ✓ 已處理完成，保留資料驗證")
        
        return True, reviewer_folder, new_filename
        
    except Exception as e:
        print(f"❌ 處理 {reviewer} 的檔案時發生錯誤: {str(e)}")
        return False, None, None

def process_reviewer_excel_windows_excel(file_path, reviewer, column_name, output_folder):
    """使用 Windows Excel COM 自動化（最可靠）"""
    try:
        reviewer_name = sanitize_folder_name(str(reviewer).strip())
        reviewer_folder = os.path.join(output_folder, reviewer_name)
        os.makedirs(reviewer_folder, exist_ok=True)
        
        base_name = os.path.basename(file_path)
        name_without_ext = os.path.splitext(base_name)[0]
        new_filename = f"{name_without_ext} - {reviewer_name}.xlsx"
        dst_path = os.path.join(reviewer_folder, new_filename)
        
        # 啟動 Excel
        excel = win32com.client.Dispatch("Excel.Application")
        excel.Visible = False
        excel.DisplayAlerts = False
        
        try:
            # 開啟原始檔案
            wb = excel.Workbooks.Open(os.path.abspath(file_path))
            ws = wb.ActiveSheet
            
            # 尋找欄位
            col_idx = None
            for col in range(1, ws.UsedRange.Columns.Count + 1):
                if ws.Cells(1, col).Value == column_name:
                    col_idx = col
                    break
            
            if not col_idx:
                raise ValueError(f"找不到欄位 '{column_name}'")
            
            # 套用篩選
            ws.AutoFilterMode = False
            ws.UsedRange.AutoFilter(Field=col_idx, Criteria1=str(reviewer))
            
            # 刪除隱藏的列
            for row in range(ws.UsedRange.Rows.Count, 1, -1):
                if ws.Rows(row).Hidden:
                    ws.Rows(row).Delete()
            
            # 移除篩選
            ws.AutoFilterMode = False
            
            # 另存新檔
            wb.SaveAs(os.path.abspath(dst_path))
            wb.Close()
            
            print(f"  ✓ 使用 Excel COM 處理完成")
            return True, reviewer_folder, new_filename
            
        finally:
            excel.Quit()
            
    except Exception as e:
        print(f"❌ Excel COM 處理失敗: {str(e)}")
        return False, None, None

print("✓ Excel 處理功能已就緒")

## 步驟 6: 使用者介面

In [ ]:
# 全域變數
last_folder = os.path.expanduser("~")
reviewer_data = {}  # 儲存審查者資料

# === SharePoint 連接介面 ===
display(HTML("<h3>🌐 步驟 1: SharePoint 連接</h3>"))

sharepoint_url_input = widgets.Text(
    value='',
    placeholder='https://company.sharepoint.com/sites/teamsite',
    description='SharePoint 網站:',
    layout=widgets.Layout(width='500px')
)

connect_button = widgets.Button(
    description='連接網站',
    layout=widgets.Layout(width='100px')
)

connection_status = widgets.HTML(value="<b>狀態:</b> 尚未連接")

def on_connect_click(b):
    """處理連接按鈕點擊"""
    global sharepoint_ctx, site_url
    with output:
        clear_output(wait=True)
        
        url = sharepoint_url_input.value.strip()
        if not url:
            print("❌ 請輸入 SharePoint 網站 URL")
            return
        
        print(f"🔍 連接到: {url}")
        print(f"👤 使用者: {current_user}")
        
        ctx = connect_sharepoint_windows_auth(url)
        
        if ctx:
            sharepoint_ctx = ctx
            site_url = url
            
            # 取得使用者資訊
            user_info = get_current_user_info(ctx)
            
            connection_status.value = (
                f"<b>狀態:</b> <span style='color:green'>✓ 已連接</span><br>"
                f"<b>使用者:</b> {user_info['title']}<br>"
                f"<b>Email:</b> {user_info['email']}"
            )
        else:
            connection_status.value = "<b>狀態:</b> <span style='color:red'>✗ 連接失敗</span>"

connect_button.on_click(on_connect_click)

display(sharepoint_url_input)
display(widgets.HBox([connect_button]))
display(connection_status)

# === 檔案選擇 ===
display(HTML("<h3>📁 步驟 2: 檔案選擇</h3>"))

def select_excel_file():
    """選擇 Excel 檔案"""
    global last_folder
    if not TKINTER_AVAILABLE:
        print("❌ 檔案對話框不可用，請手動輸入檔案路徑")
        return
    
    try:
        root = tk.Tk()
        root.withdraw()
        root.lift()
        root.attributes('-topmost', True)
        
        file_path = filedialog.askopenfilename(
            title="選擇 Excel 檔案",
            filetypes=[(
                "Excel 檔案", "*.xlsx *.xls"),
                ("所有檔案", "*.*")
            ],
            initialdir=last_folder
        )
        
        root.destroy()
        
        if file_path:
            excel_file_input.value = file_path
            last_folder = os.path.dirname(file_path)
            print(f"✓ 已選擇: {os.path.basename(file_path)}")
            
    except Exception as e:
        print(f"❌ 選擇檔案時發生錯誤: {str(e)}")

excel_file_input = widgets.Text(
    value='',
    placeholder='選擇 Excel 檔案...',
    description='Excel 檔案:',
    layout=widgets.Layout(width='450px')
)

browse_button = widgets.Button(
    description='瀏覽...',
    layout=widgets.Layout(width='100px')
)

browse_button.on_click(lambda x: select_excel_file())

reviewer_column_input = widgets.Text(
    value='Reviewer',
    description='審查者欄位:',
    layout=widgets.Layout(width='300px')
)

display(widgets.HBox([excel_file_input, browse_button]))
display(reviewer_column_input)

# === 選項設定 ===
display(HTML("<h3>⚙️ 步驟 3: 選項設定</h3>"))

copy_word_check = widgets.Checkbox(
    value=True,
    description='複製 Word 文件 (.doc, .docx)'
)

copy_pdf_check = widgets.Checkbox(
    value=True,
    description='複製 PDF 文件 (.pdf)'
)

upload_to_sharepoint_check = widgets.Checkbox(
    value=False,
    description='上傳檔案到 SharePoint'
)

# 處理方法選擇
processing_method = widgets.RadioButtons(
    options=[
        ('複製檔案後處理（推薦）', 'copy_file_first'),
        ('使用 Excel COM 自動化', 'excel_com')
    ],
    value='copy_file_first',
    description='處理方法:',
    disabled=False
)

display(copy_word_check)
display(copy_pdf_check)
display(upload_to_sharepoint_check)
display(HTML("<br><b>Excel 處理方法:</b>"))
display(processing_method)

# === 處理按鈕 ===
process_button = widgets.Button(
    description='處理 Excel 檔案',
    layout=widgets.Layout(width='200px', height='40px')
)

# === 輸出區域 ===
output = widgets.Output()
sharing_container = widgets.VBox()

display(HTML("<br>"))
display(process_button)
display(output)
display(sharing_container)

print("\n✅ 介面已就緒！")

## 步驟 7: 審查者選擇與分享介面

In [ ]:
def create_reviewer_sharing_ui(reviewers: List[str], processed_folders: Dict[str, str]):
    """建立審查者選擇與分享介面"""
    global reviewer_data
    
    # 初始化審查者資料
    reviewer_data = {}
    
    # 建立 UI 元件
    select_all = widgets.Checkbox(
        value=True,
        description='全選',
        indent=False,
        layout=widgets.Layout(width='100px')
    )
    
    reviewer_widgets = []
    
    for reviewer in reviewers:
        reviewer_str = str(reviewer).strip()
        
        # 建立勾選框
        checkbox = widgets.Checkbox(
            value=True,
            description='',
            indent=False,
            layout=widgets.Layout(width='30px')
        )
        
        # 建立名稱標籤
        name_label = widgets.Label(
            value=reviewer_str[:20] + '...' if len(reviewer_str) > 20 else reviewer_str,
            layout=widgets.Layout(width='150px')
        )
        
        # 建立 email 輸入框
        email_input = widgets.Text(
            value='',
            placeholder='輸入 email 地址',
            layout=widgets.Layout(width='250px')
        )
        
        # 建立狀態標籤
        status_label = widgets.HTML(
            value="<span style='color:orange'>⚠ 需輸入 email</span>",
            layout=widgets.Layout(width='120px')
        )
        
        # 儲存資料
        reviewer_data[reviewer_str] = {
            'checkbox': checkbox,
            'email_input': email_input,
            'status_label': status_label,
            'folder_path': processed_folders.get(reviewer_str, ''),
            'selected': True,
            'status': 'ready'
        }
        
        # 建立列
        row = widgets.HBox([
            checkbox,
            name_label,
            email_input,
            status_label
        ])
        
        reviewer_widgets.append(row)
    
    # 全選處理函數
    def on_select_all_change(change):
        for data in reviewer_data.values():
            data['checkbox'].value = change['new']
    
    select_all.observe(on_select_all_change, names='value')
    
    # 分享按鈕
    share_button = widgets.Button(
        description='分享給選定的審查者',
        layout=widgets.Layout(width='200px', height='40px')
    )
    
    # 進度條
    progress = widgets.IntProgress(
        value=0,
        min=0,
        max=len(reviewers),
        description='進度:',
        layout=widgets.Layout(width='400px')
    )
    
    progress_label = widgets.Label(value='')
    
    # 狀態輸出
    status_output = widgets.Output()
    
    # 分享按鈕處理函數
    def on_share_click(b):
        with status_output:
            clear_output()
            share_folders_to_reviewers(progress, progress_label)
    
    share_button.on_click(on_share_click)
    
    # 建立 UI
    sharing_ui = widgets.VBox([
        widgets.HTML("<h3>📤 步驟 4: SharePoint 分享</h3>"),
        widgets.HTML("<p>輸入審查者的 email 地址：</p>"),
        widgets.HBox([select_all]),
        widgets.HTML("<hr>"),
        widgets.VBox(reviewer_widgets),
        widgets.HTML("<hr>"),
        widgets.HBox([share_button]),
        widgets.HBox([progress, progress_label]),
        status_output
    ])
    
    return sharing_ui

def share_folders_to_reviewers(progress_widget, progress_label):
    """批次分享資料夾給審查者"""
    global sharepoint_ctx
    
    if not sharepoint_ctx:
        print("❌ 請先連接 SharePoint 網站")
        return
    
    # 取得選定的審查者
    selected_reviewers = []
    for reviewer, data in reviewer_data.items():
        if data['checkbox'].value:
            email = data['email_input'].value.strip()
            if email:
                selected_reviewers.append((reviewer, email, data['folder_path']))
            else:
                print(f"⚠️ 跳過 {reviewer} - 未提供 email 地址")
    
    if not selected_reviewers:
        print("❌ 沒有選擇審查者或沒有提供 email 地址")
        return
    
    print(f"\n📤 開始分享給 {len(selected_reviewers)} 位審查者...\n")
    
    # 重設進度
    progress_widget.value = 0
    progress_widget.max = len(selected_reviewers)
    
    success_count = 0
    
    for i, (reviewer, email, folder_path) in enumerate(selected_reviewers):
        progress_label.value = f"{reviewer}"
        
        # 更新狀態
        data = reviewer_data[reviewer]
        data['status_label'].value = "<span style='color:blue'>⏳ 分享中...</span>"
        
        try:
            # 分享資料夾
            folder_name = sanitize_folder_name(reviewer)
            success = share_folder_with_email(sharepoint_ctx, folder_name, email, 'Edit')
            
            if success:
                data['status_label'].value = "<span style='color:green'>✓ 已分享</span>"
                print(f"✓ {reviewer}: 資料夾已分享給 {email}")
                success_count += 1
            else:
                data['status_label'].value = "<span style='color:red'>✗ 分享失敗</span>"
                print(f"✗ {reviewer}: 分享失敗")
                
        except Exception as e:
            data['status_label'].value = "<span style='color:red'>✗ 錯誤</span>"
            print(f"✗ {reviewer}: 錯誤 - {str(e)}")
        
        # 更新進度
        progress_widget.value = i + 1
        time.sleep(0.5)  # 避免速率限制
    
    # 完成總結
    progress_label.value = "完成！"
    print(f"\n✅ 分享完成！成功分享給 {success_count}/{len(selected_reviewers)} 位審查者。")
    
    if site_url:
        print(f"\n📁 SharePoint 網站: {site_url}")
        print("審查者可以在 SharePoint 上找到他們的專屬資料夾。")

print("✓ 審查者選擇介面已就緒")

## 步驟 8: 主要處理函數

In [ ]:
def process_excel_file(button):
    """主要處理函數"""
    with output:
        clear_output()
        
        # 驗證輸入
        if not excel_file_input.value:
            print("❌ 請選擇 Excel 檔案")
            return
        
        if not sharepoint_ctx and upload_to_sharepoint_check.value:
            print("❌ 請先連接 SharePoint 網站")
            return
        
        file_path = excel_file_input.value.strip()
        column_name = reviewer_column_input.value.strip()
        method = processing_method.value
        
        if not os.path.exists(file_path):
            print(f"❌ 找不到檔案: {file_path}")
            return
        
        print(f"📁 處理中: {os.path.basename(file_path)}")
        print(f"📊 審查者欄位: {column_name}")
        print(f"🔧 處理方法: {method}")
        print("=" * 50)
        
        try:
            # 讀取 Excel 檔案
            df = pd.read_excel(file_path, engine='openpyxl')
            
            if column_name not in df.columns:
                print(f"❌ 找不到欄位 '{column_name}'")
                print(f"可用欄位: {', '.join(df.columns)}")
                return
            
            # 取得唯一審查者
            reviewers = df[column_name].dropna().unique().tolist()
            print(f"✓ 找到 {len(reviewers)} 位審查者")
            
            # 處理每位審查者
            base_dir = os.path.dirname(file_path)
            processed = 0
            processed_folders = {}
            
            for reviewer in reviewers:
                print(f"\n📝 處理中: {reviewer}")
                
                # 根據選擇的方法處理
                if method == 'excel_com':
                    success, folder_path, filename = process_reviewer_excel_windows_excel(
                        file_path, reviewer, column_name, base_dir
                    )
                else:
                    success, folder_path, filename = process_reviewer_excel_copy_first(
                        file_path, reviewer, column_name, base_dir
                    )
                
                if success:
                    print(f"  ✓ 已建立: {filename}")
                    processed_folders[str(reviewer).strip()] = folder_path
                    
                    # 複製相關文件
                    if copy_word_check.value or copy_pdf_check.value:
                        copied = copy_selected_documents(
                            base_dir, folder_path,
                            copy_word=copy_word_check.value,
                            copy_pdf=copy_pdf_check.value
                        )
                        if copied:
                            print(f"  ✓ 已複製 {len(copied)} 個文件")
                    
                    # 上傳到 SharePoint
                    if upload_to_sharepoint_check.value and sharepoint_ctx:
                        print("  📤 上傳到 SharePoint...")
                        
                        # 建立 SharePoint 資料夾
                        sp_folder = sanitize_folder_name(str(reviewer).strip())
                        if create_sharepoint_folder(sharepoint_ctx, sp_folder):
                            # 上傳 Excel 檔案
                            excel_path = os.path.join(folder_path, filename)
                            if upload_file_to_folder(sharepoint_ctx, excel_path, sp_folder):
                                print(f"    ✓ 已上傳 Excel")
                            
                            # 上傳其他文件
                            for doc in copied:
                                doc_path = os.path.join(folder_path, doc)
                                if os.path.exists(doc_path):
                                    if upload_file_to_folder(sharepoint_ctx, doc_path, sp_folder):
                                        print(f"    ✓ 已上傳: {doc}")
                    
                    processed += 1
            
            # 總結
            print("\n" + "=" * 50)
            print(f"✅ 處理完成！")
            print(f"📊 已處理 {processed}/{len(reviewers)} 位審查者")
            print(f"📁 輸出位置: {base_dir}")
            
            # 建立分享介面
            global sharing_container
            sharing_ui = create_reviewer_sharing_ui(reviewers, processed_folders)
            sharing_container.children = [sharing_ui]
            
            print("\n💡 提示：")
            print("1. 輸入審查者的 email 地址")
            print("2. 選擇要分享的審查者")
            print("3. 點擊『分享給選定的審查者』按鈕")
            
        except Exception as e:
            print(f"\n❌ 發生錯誤: {str(e)}")
            import traceback
            traceback.print_exc()

# 連接處理函數到按鈕
process_button.on_click(process_excel_file)
print("✓ 主要處理函數已就緒")

## 使用說明

### 系統需求

1. **Windows 環境**
   - 已加入公司網域的 Windows 電腦
   - 已使用公司帳號登入
   - 具有 SharePoint 存取權限

2. **軟體需求**
   - Python 3.9+
   - Microsoft Office（建議安裝）
   - 瀏覽器（用於 SharePoint 存取）

### 使用流程

1. **連接 SharePoint**
   - 輸入 SharePoint 網站 URL
   - 點擊『連接網站』
   - 系統會嘗試使用 Windows 認證自動連接
   - 如失敗，會出現登入對話框

2. **處理 Excel**
   - 選擇要處理的 Excel 檔案
   - 確認審查者欄位名稱
   - 選擇要複製的文件類型
   - 選擇處理方法（推薦使用「複製檔案後處理」）
   - 點擊『處理 Excel 檔案』

3. **分享資料夾**
   - 為每位審查者輸入 email 地址
   - 選擇要分享的審查者
   - 點擊『分享給選定的審查者』

### 處理方法說明

1. **複製檔案後處理（推薦）**
   - 先複製整個 Excel 檔案
   - 再刪除不相關的資料列
   - 保留所有資料驗證規則
   - 適用於大部分情況

2. **使用 Excel COM 自動化**
   - 使用 Windows Excel 程式處理
   - 最可靠但速度較慢
   - 需要安裝 Microsoft Excel

### 功能特色

- ✅ **Windows 整合驗證**：無需額外設定 Azure AD
- ✅ **保留資料驗證**：完整保留 Excel 資料驗證規則
- ✅ **批次處理**：一次處理多位審查者
- ✅ **自動上傳**：可選擇自動上傳到 SharePoint
- ✅ **權限管理**：自動設定資料夾分享權限

### 疑難排解

1. **連接失敗**
   - 確認已登入 Windows 網域
   - 檢查 SharePoint URL 是否正確
   - 確認有 SharePoint 存取權限

2. **資料驗證遺失**
   - 使用「複製檔案後處理」方法
   - 或使用「Excel COM 自動化」方法

3. **分享失敗**
   - 確認 email 地址正確
   - 確認有資料夾分享權限
   - 檢查使用者是否在組織內

4. **上傳失敗**
   - 檢查檔案大小限制
   - 確認網路連線穩定
   - 確認有上傳權限